<h1> 2021-2 캡스톤 디자인 </h1>  


<h3>목표 : 대출량 예측 및 우선 대출 도서 선정 모델 개발</h3>  

데이터 수집 :  
1.도서관 정보나루 API 에서 국립세종도서관의 대출 데이터 수집  
2.국립중앙도서관 Open API에서 서지정보 수집 

In [74]:
import pandas as pd
import numpy as np
import json
from urllib.request import urlopen, Request
import urllib
import requests
import re
from konlpy.tag import Okt
from gensim import models
import time
from tqdm import trange
from PIL import Image

In [6]:
jn_key = 'f8a1ee19fb60a1fe7d89fbefd5a1ee729d4a67be99b748cdef74e3b1381f9e6e'

In [36]:
isbn = []
loan_time = []


url = f'http://data4library.kr/api/itemSrch?&libCode=50001&authKey={jn_key}&pageNo={1}&pageSize=100&format=json&startDt=2020-08-31&endDt=2021-08-31'
res = urlopen(url)
res_m = res.read().decode('utf-8')
res_json = json.loads(res_m)
pc = res_json['response']['numFound'] // 100 if (res_json['response']['numFound'] % 100) == 0 else (res_json['response']['numFound'] // 100) + 1

for i in trange(1, pc + 1):
    url = f'http://data4library.kr/api/itemSrch?&libCode=50001&authKey={jn_key}&pageNo={i}&pageSize=100&format=json&startDt=2020-08-31&endDt=2021-08-31'
    res = urlopen(url)
    res_m = res.read().decode('utf-8')
    res_json = json.loads(res_m)
    for j in res_json['response']['docs']:
        isbn.append(j['doc']['isbn13'])
        loan_time.append(j['doc']['reg_date'])

100%|████████████████████████████████████████████████████████████████████████████████| 851/851 [26:38<00:00,  1.88s/it]


In [38]:
loan = pd.DataFrame({'isbn' : isbn, 'loan_time' : loan_time})
loan.to_csv('loan.csv',index = False)
loan

,isbn,loan_time
0,9791160947427,2021-08-31
1,9791191825022,2021-08-31
2,9788934989349,2021-08-31
3,9788998940010,2021-08-31
4,9791187512578,2021-08-31
...,...,...
85047,9791190582308,2020-08-31
85048,9791158511807,2020-08-31
85049,9791196572846,2020-08-31
85050,9791190242455,2020-08-31


In [37]:
oa_key = '578ca4ba507631e4a9b621f4029400eac427aaf6071b45611e599387b637b6dc'

In [68]:
author = []
publish_date = []
title = []
title_image_url = []
pages = []
price = []

reader = []
pub_type = []
subject_1 = []
subject_2 = []

isbn = loan.isbn.value_counts().index
blank_isbn = []
isbn_loop = []
for isb in trange(len(isbn)):
    i = isbn[isb] 
    url = f'http://seoji.nl.go.kr/landingPage/SearchApi.do?cert_key={oa_key}&isbn={i}&result_style=json&page_size=1&page_no=1'
    res = requests.get(url)
    try:
        info = res.json()['docs'][0]
    except:
        blank_isbn.append(i)
        continue
        
    isbn_loop.append(i)
    title.append(info['TITLE'])
    title_image_url.append(info['TITLE_URL'])
    
    if info['REAL_PUBLISH_DATE'] == '':
        publish_date.append(info['PUBLISH_PREDATE'])
    else:
        publish_date.append(info['REAL_PUBLISH_DATE'])
        
    if info['REAL_PRICE'] == '':
        price.append(info['PRE_PRICE'])
    else:
        price.append(info['REAL_PRICE'])
        
    try:
        ea_code = info['EA_ADD_CODE']
        subject_1.append(info['EA_ADD_CODE'][2])
        subject_2.append(info['EA_ADD_CODE'][3])
        reader.append(info['EA_ADD_CODE'][0])
        pub_type.append(info['EA_ADD_CODE'][1])
    except:
        subject_1.append(np.NaN)
        subject_2.append(np.NaN)
        reader.append(np.NaN)
        pub_type.append(np.NaN)
    pages.append(info['PAGE'])
    author.append(info['AUTHOR'])

100%|██████████████████████████████████████████████████████████████████████████| 42279/42279 [1:00:54<00:00, 11.57it/s]


In [ ]:
book = pd.DataFrame({'isbn' : isbn_loop, 'author' : author, 'pub_date' : publish_date, 'title' : title, 'title_image_url' : title_image_url, 'pages' : pages, 'price' :  price, 'reader' : reader, 'pub_type' : pub_type, 'subject_1' : subject_1, 'subject_2' : subject_2})
book.to_csv('book.csv', index = False)

In [62]:
book

,isbn,author,pub_date,title,title_image_url,pages,price,reader,pub_type,subject_1,subject_2
0,9788984374164,지은이: 기욤 뮈소 ;옮긴이: 양영란,20201124.0,인생은 소설이다,NaN,311 p.,15000,0.0,3.0,8.0,6.0
1,9791160953268,충청남도교육청교육연수원 [편],20190401.0,(2019) 제16기 핵심인재 양성과정,NaN,"xvi, 467 p.",비매품/무료,1.0,3.0,3.0,5.0
2,9788934986652,글:오은영 ;그림: 차상미,20201025.0,어떻게 말해줘야 할까,http://seoji.nl.go.kr/fu/ecip/dbfiles/CIP_FILE...,397 p.,17500,1.0,3.0,5.0,9.0
3,9791190808125,라곰씨 글 ;차차 그림,20210129.0,(무엇이든 해결단) 허팝 연구소,NaN,124 p.,13000,7.0,4.0,8.0,1.0
4,9791164137176,흔한남매 원작 ;김언정 글 ;차차 그림,20201215.0,(흔한남매) 별난 방탈출,NaN,126 p.,12000,7.0,7.0,8.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
37312,9788954677547,바버라 오클리 지음 ;김아림 옮김,20210302.0,이과형 두뇌 활용법,NaN,369 p.,17000,0.0,3.0,4.0,0.0
37313,9791189329068,권미정;김은주;백난희;양승옥;유율희;윤미옥;이지형;임희,20200310.0,당신의 행복은 안녕하신가요?,NaN,307 p.,20000,0.0,3.0,1.0,9.0
37314,9788997728404,글: 마크 서머셋 ; 그림: 로완 서머셋,20131126.0,똑똑해지는 약,http://seoji.nl.go.kr/fu/ecip/dbfiles/CIP_FILE...,NaN,15000,7.0,7.0,8.0,9.0
37315,9791189773175,저자 : 윤혜정,20200601.0,"민들레야, 잘가",NaN,42,"15,000",0.0,7.0,8.0,0.0


In [46]:
book.title_image_url.value_counts()

http://seoji.nl.go.kr/fu/ecip/dbfiles/CIP_FILES_TBL/2019/04/19/9788952799432.JPG    1
http://seoji.nl.go.kr/fu/ecip/dbfiles/CIP_FILES_TBL/2020/09/21/9791185788227.jpg    1
http://seoji.nl.go.kr/fu/ecip/dbfiles/CIP_FILES_TBL/2020/12/21/9788998153465.jpg    1
http://seoji.nl.go.kr/fu/ecip/dbfiles/CIP_FILES_TBL/2020/11/15/9791166340017.jpg    1
http://seoji.nl.go.kr/fu/ecip/dbfiles/CIP_FILES_TBL/2019/11/18/9791159314377.jpg    1
                                                                                   ..
http://seoji.nl.go.kr/fu/ecip/dbfiles/CIP_FILES_TBL/2020/09/07/9791186871805.jpg    1
http://seoji.nl.go.kr/fu/ecip/dbfiles/CIP_FILES_TBL/2020/10/26/9791188352395.jpg    1
http://seoji.nl.go.kr/fu/ecip/dbfiles/CIP_FILES_TBL/2020/11/10/9791165512446.jpg    1
http://seoji.nl.go.kr/fu/ecip/dbfiles/CIP_FILES_TBL/2019/12/23/9791196100308.jpg    1
http://seoji.nl.go.kr/fu/ecip/dbfiles/CIP_FILES_TBL/2020/09/11/9791165367947.jpg    1
Name: title_image_url, Length: 10397, dtype: int64

In [60]:
no_image = book.loc[book.title_image_url.isna(), ]
isbn = no_image.isbn
title_image_url = []

for isb in trange(len(isbn)):
    try:
        i = isbn.iloc[isb]
        url = f'http://data4library.kr/api/srchDtlList?authKey={jn_key}&isbn13={i}&format=json'
        res = requests.get(url)
        title_image_url.append(res.json()['response']['detail'][0]['book']['bookImageURL'])
    except:
        title_image_url.append(np.NaN)

100%|██████████████████████████████████████████████████████████████████████████| 26920/26920 [2:25:43<00:00,  3.08it/s]


In [67]:
for isb in trange(len(isbn)):
    i = isbn.iloc[isb]
    book.loc[book.isbn == i,'title_image_url'] = title_image_url[isb]

100%|███████████████████████████████████████████████████████████████████████████| 26920/26920 [01:03<00:00, 426.13it/s]


In [69]:
book.title_image_url.value_counts()

                                                                                         7167
https://image.aladin.co.kr/product/25836/43/cover/8949105292_1.jpg                          2
https://image.aladin.co.kr/product/24936/86/cover/8937472805_1.jpg                          1
https://image.aladin.co.kr/product/26122/39/cover/8965642604_1.jpg                          1
https://image.aladin.co.kr/product/25243/46/cover/8901244152_1.jpg                          1
                                                                                         ... 
https://bookthumb-phinf.pstatic.net/cover/161/251/16125112.jpg?type=m1&udate=20200122       1
http://seoji.nl.go.kr/fu/ecip/dbfiles/CIP_FILES_TBL/2020/11/05/9788996897262.jpg            1
https://image.aladin.co.kr/product/26094/6/cover/k712737250_1.jpg                           1
http://seoji.nl.go.kr/fu/ecip/dbfiles/CIP_FILES_TBL/2021/01/19/9791160805475.jpg            1
http://seoji.nl.go.kr/fu/ecip/dbfiles/CIP_FILES_TBL/2020/09/

In [90]:
for isb in trange(len(isbn)):
    image_link = book.title_image_url.iloc[isb]
    if (image_link == '' ) | (np.isnan(image_link)):
        continue
    else:
        urllib.request.urlretrieve(image_link, f'C:/Users/soymi/Desktop/lib_git/2021-2Capstonedesign/image/{book.isbn.iloc[isb]}.jpg',)

  0%|                                                                                        | 0/26920 [00:00<?, ?it/s]


TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [79]:
url = 'https://image.aladin.co.kr/product/25836/43/cover/8949105292_1.jpg'

i = 1

urllib.request.urlretrieve(url, f'C:/Users/soymi/Desktop/lib_git/2021-2Capstonedesign/image/{i}.jpg',)

('C:/Users/soymi/Desktop/lib_git/2021-2Capstonedesign/image/1.jpg',
 <http.client.HTTPMessage at 0x1bd151563a0>)

In [117]:
only_image = (book.loc[pd.DataFrame({'col1' : book.title_image_url.notna(),'col2' : book.title_image_url != ''}).all(axis = 1), ['isbn','title_image_url']]).reset_index(drop=True)
image_isbn = []

for i in trange(len(only_image)):
    try:
        urllib.request.urlretrieve(only_image.iloc[i,1], f'C:/Users/soymi/Desktop/lib_git/2021-2Capstonedesign/image/{only_image.iloc[i,0]}.jpg',)
        image_isbn.append(only_image.iloc[i,0])
    except:
        continue

100%|██████████████████████████████████████████████████████████████████████████| 30136/30136 [1:24:21<00:00,  5.95it/s]


In [115]:
only_image.iloc[i,1]

'https://image.aladin.co.kr/product/26076/92/cover/k862737153_1.jpg'